# Notebook de referência

Nome:

## Instruções:


Treinar e medir a acurácia de um modelo BERT (ou variantes) para classificação binária usando o dataset do IMDB (20k/5k amostras de treino/validação).

Importante:
- Deve-se implementar o próprio laço de treinamento.
- Implementar o acumulo de gradiente.

Dicas:
- BERT geralmente costuma aprender bem uma tarefa com poucas épocas (de 3 a 5 épocas). Se tiver demorando mais de 5 épocas para chegar em 80% de acurácia, ajuste os hiperparametros.

- Solução para erro de memória:
  - Usar bfloat16 permite quase dobrar o batch size

Opcional:
- Pode-se usar a função trainer da biblioteca Transformers/HuggingFace para verificar se seu laço de treinamento está correto. Note que ainda assim é obrigatório implementar o laço próprio.

# Fixando a seed

In [ ]:
import random
import torch
import torch.nn.functional as F
import numpy as np

In [ ]:
random.seed(123)
np.random.seed(123)
torch.manual_seed(123)

## Preparando Dados

Primeiro, fazemos download do dataset:

In [ ]:
import os

# extract tar.gz
import tarfile


def unzip_tar_gz(file_path, dest_dir):
    with tarfile.open(file_path, "r:gz") as tar:
        tar.extractall(path=dest_dir)


if not os.path.exists("data/aclImdb"):

    os.makedirs("data", exist_ok=True)

    # Download the dataset
    !wget -nc http://ai.stanford.edu/~amaas/data/sentiment/aclImdb_v1.tar.gz -P data

    # Extract the dataset
    unzip_tar_gz("data/aclImdb_v1.tar.gz", "data")

## Carregando o dataset

Criaremos uma divisão de treino (20k exemplos) e validação (5k exemplos) artificialmente.

In [4]:
import os

max_valid = 5000


def load_texts(folder):
    texts = []
    for path in os.listdir(folder):
        with open(
            os.path.join(folder, path), encoding="utf-8"
        ) as f:  # Especificando o codec como UTF-8
            texts.append(f.read())
    return texts


x_train_pos = load_texts("data/aclImdb/train/pos")
x_train_neg = load_texts("data/aclImdb/train/neg")
x_test_pos = load_texts("data/aclImdb/test/pos")
x_test_neg = load_texts("data/aclImdb/test/neg")

x_train = x_train_pos + x_train_neg
x_test = x_test_pos + x_test_neg
y_train = [1] * len(x_train_pos) + [0] * len(x_train_neg)
y_test = [1] * len(x_test_pos) + [0] * len(x_test_neg)

# Embaralhamos o treino para depois fazermos a divisão treino/valid.
c = list(zip(x_train, y_train))
random.shuffle(c)
x_train, y_train = zip(*c)

x_valid = x_train[-max_valid:]
y_valid = y_train[-max_valid:]
x_train = x_train[:-max_valid]
y_train = y_train[:-max_valid]

print(len(x_train), "amostras de treino.")
print(len(x_valid), "amostras de desenvolvimento.")
print(len(x_test), "amostras de teste.")

print("3 primeiras amostras treino:")
for x, y in zip(x_train[:3], y_train[:3]):
    print(y, x[:100])

print("3 últimas amostras treino:")
for x, y in zip(x_train[-3:], y_train[-3:]):
    print(y, x[:100])

print("3 primeiras amostras validação:")
for x, y in zip(x_valid[:3], y_test[:3]):
    print(y, x[:100])

print("3 últimas amostras validação:")
for x, y in zip(x_valid[-3:], y_valid[-3:]):
    print(y, x[:100])

### Model

In [ ]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification

n_classes = len(set(y_train))

tokenizer = AutoTokenizer.from_pretrained("sagui-nlp/debertinha-ptbr-xsmall")
model = AutoModelForSequenceClassification.from_pretrained(
    "sagui-nlp/debertinha-ptbr-xsmall", num_labels=n_classes
)

### Tokenization

In [ ]:
max_lenght = model.config.max_position_embeddings

x_train_tokenized = tokenizer(
    x_train, padding=True, truncation=True, max_length=max_lenght, return_tensors="pt"
)

x_valid_tokenized = tokenizer(
    x_valid, padding=True, truncation=True, max_length=max_lenght, return_tensors="pt"
)

x_test_tokenized = tokenizer(
    x_test, padding=True, truncation=True, max_length=max_lenght, return_tensors="pt"
)

### Dataset

In [ ]:
from datasets import Dataset

train_dataset = Dataset.from_dict({**x_train_tokenized, "label": y_train})
valid_dataset = Dataset.from_dict({**x_valid_tokenized, "label": y_valid})
test_dataset = Dataset.from_dict({**x_test_tokenized, "label": y_test})

train_dataset.set_format(
    "pt", columns=["input_ids", "attention_mask", "label"], output_all_columns=True
)
valid_dataset.set_format(
    "pt", columns=["input_ids", "attention_mask", "label"], output_all_columns=True
)
test_dataset.set_format(
    "pt", columns=["input_ids", "attention_mask", "label"], output_all_columns=True
)

### Data Loader

In [ ]:
batch_size = 2
train_dataloader = torch.utils.data.DataLoader(
    train_dataset,
    batch_size=batch_size,
    shuffle=True,
)
valid_dataloader = torch.utils.data.DataLoader(valid_dataset, batch_size=batch_size)
test_dataloader = torch.utils.data.DataLoader(test_dataset, batch_size=batch_size)

In [ ]:
batch = next(iter(train_dataloader))

### Train

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [ ]:
epochs = 5
optimizer = torch.optim.AdamW(model.parameters(), lr=5e-5)
criterion = torch.nn.CrossEntropyLoss()
accumulation_steps = 4

model.to(device)

In [ ]:
accumulation_steps = 4  # Defina o número de passos de acumulação desejados

for epoch in range(epochs):
    model.train()
    total_loss = 0
    for step, batch in enumerate(train_dataloader):
        optimizer.zero_grad()
        input_ids = batch["input_ids"].to(device)
        attention_mask = batch["attention_mask"].to(device)
        labels = batch["label"].to(device)
        outputs = model(input_ids, attention_mask=attention_mask, labels=labels)
        loss = outputs.loss
        loss = (
            loss / accumulation_steps
        )  # Divida a perda pelo número de passos de acumulação
        loss.backward()

        if (step + 1) % accumulation_steps == 0 or step == len(train_dataloader) - 1:
            optimizer.step()
            optimizer.zero_grad()  # Zera os gradientes apenas depois de fazer a atualização

        total_loss += loss.item()

    model.eval()
    with torch.no_grad():
        total_loss_valid = 0
        total = 0
        correct = 0
        for batch in valid_dataloader:
            input_ids = batch["input_ids"].to(device)
            attention_mask = batch["attention_mask"].to(device)
            labels = batch["label"].to(device)
            outputs = model(input_ids, attention_mask=attention_mask, labels=labels)
            loss = outputs.loss
            total_loss_valid += loss.item()
            total += len(labels)
            correct += (outputs.logits.argmax(dim=1) == labels).sum().item()

    print(
        f"Epoch {epoch+1}/{epochs} - Train Loss: {total_loss/len(train_dataloader):.4f} - Valid Loss: {total_loss_valid/len(valid_dataloader):.4f} - Accuracy: {correct/total:.4f}"
    )